# ANTs Registration Workflow

In [ ]:
import os, glob, time, pydicom
import SimpleITK as sitk
import numpy as np
from rt_utils import RTStructBuilder

### Set input file locations

In [ ]:
mrpath = glob.glob('/software/notebooks/MR*')[0]
t2path = glob.glob(os.path.join(mrpath,'S0301*'))[0]
#roiniifile = glob.glob(os.path.join(mrpath,'301_*ROI.nii.gz'))[0]

fmisopath = glob.glob('/software/notebooks/FMISO*')[0]
ctpath = glob.glob(os.path.join(fmisopath,'S0002_CT-*'))[0]

In [ ]:
coregpath = '/software/notebooks/coreg_20230130'
os.makedirs(coregpath, exist_ok = True)

moving_nii = os.path.join(coregpath, 't2moving.nii.gz')
fixed_nii = os.path.join(coregpath, 'ctfixed.nii.gz')

### Convert DICOM to NIfTI

In [ ]:
def dcm2nii(dcmpath,savenii):
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(dcmpath)
    reader.SetFileNames(dicom_names)
    dcm_img = reader.Execute()
    writer = sitk.ImageFileWriter()
    writer.SetFileName(savenii)
    writer.Execute(sitk.Cast(dcm_img,sitk.sitkFloat32))

In [ ]:
dcm2nii(t2path, moving_nii)
dcm2nii(ctpath,fixed_nii)

### N4 Correct and Coregister

In [ ]:
#define filename for N4-corrected MR
n4_nii = os.path.join(coregpath,'n4_t2moving.nii.gz')

#transform type is rigid-only
transform_type = 'r'

#prefix of final coregistred files
out_nii_prefix = os.path.join(coregpath,transform_type + '_n4MR_to_CT_')
out_nii = out_nii_prefix + 'Warped.nii.gz'

In [ ]:
%%bash -s {fixed_nii} {moving_nii} {n4_nii} {transform_type} {out_nii_prefix}

N4BiasFieldCorrection -d 3 -i $2 -o $3 
antsRegistrationSyNQuick.sh -d 3 -f $1 -m $3 -t $4 -o $5 -n 4

In [ ]:
# Function output nii to DICOM

def nii2dcm(coregniifile, fixed_dcmpath, moving_dcmpath, dcm_out, coreg_seriesdescription = 'Coregistered Image'):      
    nii_img = sitk.Cast(sitk.ReadImage(coregniifile),sitk.sitkInt32)
    
    #Convert cropped full FOV CB nii to DICOM
    series_IDs = sitk.ImageSeriesReader.GetGDCMSeriesIDs(fixed_dcmpath)
    series_file_names = sitk.ImageSeriesReader.GetGDCMSeriesFileNames(fixed_dcmpath, series_IDs[0])

    fixed_series_reader = sitk.ImageSeriesReader()
    
    fixed_series_reader.SetFileNames(series_file_names)
    fixed_series_reader.MetaDataDictionaryArrayUpdateOn()
    fixed_series_reader.LoadPrivateTagsOn()
    ref_fixed_img = fixed_series_reader.Execute()
    
    series_IDs = sitk.ImageSeriesReader.GetGDCMSeriesIDs(moving_dcmpath)
    series_file_names = sitk.ImageSeriesReader.GetGDCMSeriesFileNames(moving_dcmpath, series_IDs[0])
    
    moving_series_reader = sitk.ImageSeriesReader()
    moving_series_reader.SetFileNames(series_file_names)
    moving_series_reader.MetaDataDictionaryArrayUpdateOn()
    moving_series_reader.LoadPrivateTagsOn()
    ref_moving_img = moving_series_reader.Execute()
    
    writer = sitk.ImageFileWriter()
    # Use the study/series/frame of reference information given in the meta-data
    # dictionary and not the automatically generated information from the file IO
    writer.KeepOriginalImageUIDOn()

    # Copy relevant tags from the original meta-data dictionary (private tags are also
    # accessible).
    fixed_tags_to_copy = ["0010|0010", # Patient Name
                "0010|0020", # Patient ID
                "0010|0030", # Patient Birth Date
                "0020|000D", # Study Instance UID, for machine consumption
                "0020|0010", # Study ID, for human consumption
                "0008|0020", # Study Date
                "0008|0030", # Study Time
                "0008|0050"  # Accession Number
                
    ]
    
    moving_tags_to_copy = ["0008|0060", # Modality
                "0008|0016"  # SOPInstanceUID
    ]
    
    #output cropped CBCT
    modification_time = time.strftime("%H%M%S")
    modification_date = time.strftime("%Y%m%d") 

    # Copy some of the tags and add the relevant tags indicating the change.
    # For the series instance UID (0020|000e), each of the components is a number, cannot start
    # with zero, and separated by a '.' We create a unique series ID using the date and time.
    # tags of interest:
    SUID_base =  '.'.join(moving_series_reader.GetMetaData(0,"0020|000e").split('.')[:-1]) + '.' + modification_date + modification_time
    direction = nii_img.GetDirection()
    series_tag_values = [(k, fixed_series_reader.GetMetaData(0,k)) for k in fixed_tags_to_copy if fixed_series_reader.HasMetaDataKey(0,k)] + \
                 [(k, moving_series_reader.GetMetaData(0,k)) for k in moving_tags_to_copy if moving_series_reader.HasMetaDataKey(0,k)] + \
                 [("0008|0031",modification_time), # Series Time
                  ("0008|0021",modification_date), # Series Date
                  ("0008|0008","DERIVED\\SECONDARY"), # Image Type
                  ("0020|000e",SUID_base), # Series Instance UID
                  ("0020|0037", '\\'.join(map(str, (direction[0], direction[3], direction[6],# Image Orientation (Patient)
                                                    direction[1],direction[4],direction[7])))),
                  ("0008|103e", coreg_seriesdescription)] # Series Description
    
    output_modality = moving_series_reader.GetMetaData(0,"0008|0060")
    os.makedirs(dcm_out,exist_ok=True)
    for i in range(nii_img.GetDepth()):
        image_slice = nii_img[:,:,i]
        # Tags shared by the series.
        for tag, value in series_tag_values:
            image_slice.SetMetaData(tag, value)
        # Slice specific tags.
        image_slice.SetMetaData("0008|0012", time.strftime("%Y%m%d")) # Instance Creation Date
        image_slice.SetMetaData("0008|0013", time.strftime("%H%M%S")) # Instance Creation Time
        image_slice.SetMetaData("0020|0032", '\\'.join(map(str,nii_img.TransformIndexToPhysicalPoint((0,0,i))))) # Image Position (Patient)
        image_slice.SetMetaData("0020,0013", str(i)) # Instance Number
        # Write to the output directory and add the extension dcm, to force writing in DICOM format.
        writer.SetFileName(os.path.join(dcm_out,output_modality + '_' + SUID_base + '_' + str(i).zfill(3) + '.dcm'))
        writer.Execute(image_slice)

In [ ]:
# Convert warped N4-corrected output to DICOM via plastimatch
outpath = os.path.join(coregpath,'movingdcm')
nii2dcm(out_nii, ctpath, t2path, outpath, 'Coregistered MR to CT')

### Warp ROI to CT space

In [ ]:
tform = out_nii_prefix + '0GenericAffine.mat'
itktform = out_nii_prefix + '0GenericAffine.txt'
print(tform)
roipath = glob.glob('/software/notebooks/coreg/301-*ROI.nii.gz')
roi_nii = os.path.basename(roipath)
out_roi = os.path.join(coregpath,transform_type + roi_nii)
print(out_roi)

In [ ]:
%%bash -s {fixed_nii} {tform} {roi_nii} {out_roi}

antsApplyTransforms -d 3 -t $2 -r $1 -i $3 -o $4 -n NearestNeighbor 

In [ ]:
# Convert warped ROI to RTSTRUCT
def roi2rtstruct(roinii,roiname,dcm_out,ref_dcm):
    #convert RT struct
    label_img = sitk.ReadImage(roinii)
    roi_arr = sitk.GetArrayFromImage(label_img) == 1
    
    rtstructout = os.path.join(dcm_out,'RTSTRUCT.dcm')
    rtstruct = RTStructBuilder.create_new(dicom_series_path = ref_dcm)
    rtstruct.add_roi(mask = np.transpose(roi_arr,(1,2,0)), color = [255,0,0], name=roiname)
    rtstruct.save(rtstructout)

    #load first cbct dicom to check FORUID in RTSTRUCT
    refhdr = pydicom.dcmread(glob.glob(os.path.join(ref_dcm,'*.dcm'))[0])
    forUID = refhdr['FrameOfReferenceUID'].value
    rt = pydicom.dcmread(rtstructout)
    rt_id_str = 'coregistered ' + roiname
    rt.add_new([0x3006,0x4],'LO',rt_id_str)
    rt[0x3006,0x20][0][0x3006,0x28].value = rt_id_str
    rt[0x8,0x103e].value = rt_id_str
    if rt[0x3006,0x10][0][0x20,0x52].value != forUID:
        rt[0x3006,0x10][0][0x20,0x52].value = forUID
        rt[0x3006,0x20][0][0x3006,0x24].value = forUID
    rt.save_as(rtstructout)

In [ ]:
roi2rtstruct(out_roi,'node',coregpath,ctpath)

In [ ]:
roi2rtstruct(roi_nii, 'node_T2space', t2path, t2path)

### Apply ANTs transform to DICOM with Plastimatch

In [ ]:
%%bash -s {tform} {itktform}

ConvertTransformFile 3 $1 $2

In [ ]:
moving_dcm = t2path
print(moving_dcm)
moving_rtstruct = '/software/notebooks/MR_D2019_11_12/S0301_RTSTRUCT.dcm'
out_dcm = '/software/notebooks/MR_D2019_11_12/coregistered_dcm'

In [ ]:
%%bash -s {itktform} "{moving_dcm}" {moving_rtstruct} {out_dcm}
echo $2

plastimatch convert --xf $1 --input "$2" --output-dicom $4
plastimatch convert --xf $1 --input $3 --output-dicom $4